In [1]:
from openai import OpenAI

# filepath: /path/to/your/file.py
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Get the OpenAI API key
openai_api_key = os.getenv('OPENAI_API_KEY')
client = OpenAI()

In [2]:


client.files.create(
  file=open("dad_joke_reasoning_samples_test.jsonl", "rb"),
  purpose="fine-tune"
)

client.files.create(
  file=open("dad_joke_reasoning_samples_train.jsonl", "rb"),
  purpose="fine-tune"
)

FileObject(id='file-XZkorTai3cwNRvj4t3Eaap', bytes=66558, created_at=1735407580, filename='dad_joke_reasoning_samples_train.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

In [3]:
files = client.files.list(purpose="fine-tune")

for file in files:
    print(file.filename.upper(),':',file.id)

DAD_JOKE_REASONING_SAMPLES_TRAIN.JSONL : file-XZkorTai3cwNRvj4t3Eaap
DAD_JOKE_REASONING_SAMPLES_TEST.JSONL : file-1fwP5mJBAVSJm1enXNHvWC
DAD_JOKE_SAMPLES_TEST.JSONL : file-14gtA38iuJ7sJgqBmwsyNC
DAD_JOKE_SAMPLES_TRAIN.JSONL : file-Jp47nPA2Eq2S9p7VXCEsWz


In [6]:
client.fine_tuning.jobs.create(
  training_file="file-XZkorTai3cwNRvj4t3Eaap",
  validation_file ="file-1fwP5mJBAVSJm1enXNHvWC",
  model="gpt-4o-2024-08-06",
  suffix="cold-warrior-reasoning"
)

FineTuningJob(id='ftjob-2ATXiHjoeP0guwRzLG9QPjLW', created_at=1735409095, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(batch_size='auto', learning_rate_multiplier='auto', n_epochs='auto'), model='gpt-4o-2024-08-06', object='fine_tuning.job', organization_id='org-RuBNDg5GaMqiVIMAjwK7becX', result_files=[], seed=834169983, status='validating_files', trained_tokens=None, training_file='file-XZkorTai3cwNRvj4t3Eaap', validation_file='file-1fwP5mJBAVSJm1enXNHvWC', estimated_finish=None, integrations=[], method=Method(dpo=None, supervised=MethodSupervised(hyperparameters=MethodSupervisedHyperparameters(batch_size='auto', learning_rate_multiplier='auto', n_epochs='auto')), type='supervised'), user_provided_suffix='cold-warrior-reasoning')

In [7]:
import time


while True:
    events = client.fine_tuning.jobs.list_events(fine_tuning_job_id="ftjob-2ATXiHjoeP0guwRzLG9QPjLW", limit=10)
    message = events.data[0].message
    print(message)
    if message == "The job has successfully completed":
        break
    

    time.sleep(20)

Validating training file: file-XZkorTai3cwNRvj4t3Eaap and validation file: file-1fwP5mJBAVSJm1enXNHvWC
Validating training file: file-XZkorTai3cwNRvj4t3Eaap and validation file: file-1fwP5mJBAVSJm1enXNHvWC
Validating training file: file-XZkorTai3cwNRvj4t3Eaap and validation file: file-1fwP5mJBAVSJm1enXNHvWC
Fine-tuning job started
Fine-tuning job started
Fine-tuning job started
Fine-tuning job started
Fine-tuning job started
Fine-tuning job started
Fine-tuning job started
Fine-tuning job started
Step 9/144: training loss=1.47
Step 19/144: training loss=1.32
Step 28/144: training loss=1.46
Step 39/144: training loss=1.25
Step 43/144: training loss=1.27
Step 53/144: training loss=0.92
Step 60/144: training loss=0.87, validation loss=1.02
Step 70/144: training loss=1.04, validation loss=1.14
Step 80/144: training loss=0.93, validation loss=1.08
Step 90/144: training loss=0.99, validation loss=1.37
Step 96/144: training loss=0.86, full validation loss=1.09
Step 103/144: training loss=0.95


In [56]:
# Retrieve the state of a fine-tune
dad_joke_model = client.fine_tuning.jobs.retrieve("ftjob-34rI6p35wsPEGLjrK6R1pUCp").fine_tuned_model
print(dad_joke_model)

ft:gpt-4o-mini-2024-07-18:personal::AjAj8dcn


In [55]:
def get_response(messages):
    response = client.chat.completions.create(
        model=dad_joke_model,
        messages=messages
    )
    return response.choices[0].message.content

def main():
    print("Welcome to the Dad Joke Chatbot! Type 'exit' to quit.")
    messages = [{"role": "system", "content": "You are a dad joke expert. Create dad jokes in accordance with the user input."}]
    while True:
        user_input = input("You: ")
        if user_input.lower() == 'exit':
            break
        messages.append({"role": "user", "content": user_input})
        response = get_response(messages=messages)
        messages.append({"role": "assistant", "content": response})
        print(f"Dad Joke Bot: {response}")


if __name__ == "__main__":
    main()

Welcome to the Dad Joke Chatbot! Type 'exit' to quit.
Dad Joke Bot: Bir baba çocuğuna düğünde herkesin yaptığı dansa ne denir? diye sormuş.
Dad Joke Bot: Gelin Hayatından Emin Midir.
